In [1]:
from sqlalchemy import create_engine
import pandas as pd
import json
import requests
import os
import time
from glob import glob
import threading
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import random
import numpy as np

In [2]:
# 取得 Token , 設置請求參數 (大陸)
url = "https://login.xiaoshouyi.com/auc/oauth2/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}
data = {
    "grant_type": "password",
    "client_id": "473f31e3a6cb9cafaec3ddf36d20092b",
    "client_secret": "ddfed65f5aeebacd0f54eaa464fcb043",
    "redirect_uri": "https://api-scrm.xiaoshouyi.com/",
    "username": "11021041@twkd.com",
    "password": "Marcellin01oiH88cCI"
}

# 發送 post 請求
response = requests.post(url, headers=headers, data=data)

if response.status_code == 200:
    print(response.json())
else:
    print("請求失敗, 狀態碼:", response.status_code)

content = response.json()
ac_token = content["access_token"]


print(ac_token)

{'access_token': 'a8f6fbcb296ff96dcdb898ea0745a1ebfdc81e46c06f9fbb34bb808f8cee7ebf.MjM2OTI2Njk5ODU2MzExMw==0', 'token_type': 'Bearer', 'expires_in': 86399, 'scope': 'all prm crm', 'tenant_id': '2369266998563113', 'login_type': '', 'encryption_key': '', 'client_id': '473f31e3a6cb9cafaec3ddf36d20092b', 'passport_id': '3052172005298442', 'mobile_params': '', 'instance_uri': 'api-scrm.xiaoshouyi.com'}
a8f6fbcb296ff96dcdb898ea0745a1ebfdc81e46c06f9fbb34bb808f8cee7ebf.MjM2OTI2Njk5ODU2MzExMw==0


In [3]:
'''
select from user
'''
url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
} 
data = {
    "xoql": "select id, name, dimDepart from user",
    #"xoql": "insert into contact(contactName, phone, email) values('貴夫人', '0912345678', 'kkdd@gmail.com')"
    "batchCount": 2000
}
response2 = requests.post(url_2, headers=headers, data = data)
ownerId = response2.json()
print(ownerId)
user_df = pd.json_normalize(ownerId["data"]["records"])  
# user_df[user_df['name']=='袁芳']


{'code': '200', 'msg': '', 'errorInfo': None, 'debugInfo': None, 'data': {'totalSize': 630, 'count': 630, 'records': [{'id': '2369301140133416', 'name': 'CRM管理员', 'dimDepart': '2586653553808230'}, {'id': '2508839248500196', 'name': '谢敏', 'dimDepart': '2369301140133408'}, {'id': '2508849255192015', 'name': '唐雪梅', 'dimDepart': '2746547845976859'}, {'id': '2531364166931931', 'name': '俞丽', 'dimDepart': '2369301140133408'}, {'id': '2531364279604701', 'name': '宋冀泽', 'dimDepart': '2369301140133408'}, {'id': '2586658357285641', 'name': '徐玮珍', 'dimDepart': '2586637755765606'}, {'id': '2586659192656747', 'name': '杨卫卫', 'dimDepart': '2586653553808212'}, {'id': '2586660964537152', 'name': '金海洋', 'dimDepart': '2586653553808230'}, {'id': '2586661145334617', 'name': '教育训练', 'dimDepart': '2746546870686577'}, {'id': '2586661881402202', 'name': '蔡家维', 'dimDepart': '2369301140133408'}, {'id': '2586663184339728', 'name': '刘俐君', 'dimDepart': '2586649840522022'}, {'id': '2586670417466176', 'name': '马梦文', 'd

In [ ]:
# from datetime import datetime

# # Unix 时间戳（以毫秒为单位）
# timestamp_ms = 1708444800000

# # 将毫秒转换为秒
# timestamp_s = timestamp_ms / 1000

# # 使用 datetime.fromtimestamp() 方法将 Unix 时间戳转换为日期
# date = datetime.fromtimestamp(timestamp_s)

# print(date)


In [ ]:
# from datetime import datetime

# # 输入日期
# date_string = "2024-03-06"
# date_format = "%Y-%m-%d"

# # 将日期字符串解析为 datetime 对象
# date = datetime.strptime(date_string, date_format)

# # 将 datetime 对象转换为 Unix 时间戳（以毫秒为单位）
# timestamp_ms = int(date.timestamp() * 1000)

# print(timestamp_ms)


In [ ]:
# '''
# select from Tasks withdraw 
# 將超過期望完成日七天的K大電訪交辦撤回重新提交
# '''
# url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
# headers = {
#     "Authorization": f"Bearer {ac_token}",
#     "Content-Type":"application/x-www-form-urlencoded"
# }
# queryLocator = ''
# Tasks_df = pd.DataFrame()

# date_to_convert = datetime(2024, 3, 19) #設定日期
# timestamp = int(date_to_convert.timestamp() * 1000)

# while True:
#     data = {
#         "xoql": f'''
#         SELECT 
#         id,
#         name,
#         customItem10__c, 
#         customItem3__c,
#         customItem8__c,
#         customItem1__c 邀約日期, 
#         customItem58__c,
#         customItem50__c, 
#         customItem42__c, 
#         entityType,
#         dimDepart,
#         dimDepart.departName 區域,
#         approvalStatus,
#         customItem11__c
#         FROM 
#             customEntity14__c
#         WHERE 
#             entityType = '3074631761565155' 
#         AND 
#             customItem1__c >= {timestamp}
#         AND
#             customItem10__c = '3147232434771494'
#         ''',
#         "batchCount": 2000,
#         "queryLocator": queryLocator
#     }
#     response = requests.post(url_2, headers=headers, data=data)
#     crm = response.json()
#     data = pd.DataFrame(crm["data"]["records"])
#     Tasks_df = pd.concat([Tasks_df, data], ignore_index=True, sort=False)
    
#     if not crm['queryLocator']:
#         break
#     queryLocator = crm['queryLocator']

# Tasks_df['approvalStatus'] = Tasks_df['approvalStatus'].astype(str)
# Tasks_df['customItem8__c'] = Tasks_df['customItem8__c'].astype(str)
# Tasks_df['customItem3__c'] = Tasks_df['customItem3__c'].astype(str)

# Tasks_df['邀約日期'] = Tasks_df['邀約日期'].astype(float)
# Tasks_df['邀約日期'] = Tasks_df['邀約日期'].apply(lambda x: pd.to_datetime(x / 1000.0, unit='s', utc=True))
# Tasks_df['邀約日期'] = Tasks_df['邀約日期'].dt.tz_convert('Asia/Taipei')
# Tasks_df['邀約日期'] = Tasks_df['邀約日期'].dt.strftime('%Y-%m-%d')
# Tasks_df['邀約日期'] = pd.to_datetime(Tasks_df['邀約日期'], errors='coerce').dt.date

# Tasks_df = Tasks_df.loc[~Tasks_df['customItem3__c'].isin(["未接听"])] 
# Tasks_df = Tasks_df.loc[~Tasks_df['customItem3__c'].isin(["K大-客户未上线"])] 
# Tasks_df = Tasks_df[~Tasks_df['approvalStatus'].str.contains('审批通过')]
# Tasks_df

In [27]:
'''
select from Tasks withdraw 
將超過期望完成日七天的K大電訪交辦撤回重新提交
'''
url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
}
queryLocator = ''
Tasks_df = pd.DataFrame()

date_to_convert = datetime(2024, 3, 19) #設定日期
timestamp = int(date_to_convert.timestamp() * 1000)

while True:
    data = {
        "xoql": f'''
        SELECT 
        id,
        name,
        customItem10__c, 
        customItem3__c,
        customItem8__c,
        customItem1__c 邀約日期, 
        customItem58__c,
        customItem50__c, 
        customItem42__c, 
        entityType,
        dimDepart,
        dimDepart.departName 區域,
        approvalStatus,
        customItem11__c
        FROM 
            customEntity14__c
        WHERE 
            entityType = '3074631761565155'
        AND
            customItem42__c != 2835550546744850
        AND 
            customItem1__c >= {timestamp}
        ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    Tasks_df = pd.concat([Tasks_df, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']

Tasks_df['customItem8__c']= Tasks_df['customItem8__c'].astype(str)
Tasks_df = Tasks_df.loc[Tasks_df['customItem8__c'].str.contains("等待")]  

Tasks_df['邀約日期'] = Tasks_df['邀約日期'].astype(float)
Tasks_df['邀約日期'] = Tasks_df['邀約日期'].apply(lambda x: pd.to_datetime(x / 1000.0, unit='s', utc=True))
Tasks_df['邀約日期'] = Tasks_df['邀約日期'].dt.tz_convert('Asia/Taipei')
Tasks_df['邀約日期'] = Tasks_df['邀約日期'].dt.strftime('%Y-%m-%d')
Tasks_df['邀約日期'] = pd.to_datetime(Tasks_df['邀約日期'], errors='coerce').dt.date

Tasks_df['customItem58__c'] = Tasks_df['customItem58__c'].astype(str)
Tasks_df['customItem3__c'] = Tasks_df['customItem3__c'].astype(str)
# Tasks_df['customItem115__c'] = '1' # 業務行為設定為"電訪"

# 已邀但時間未定
Tasks_df2 = Tasks_df.loc[Tasks_df['customItem58__c'].str.contains("已邀") | Tasks_df['customItem3__c'].str.contains("已邀")] 
Tasks_df2 = Tasks_df2.loc[~(Tasks_df2['customItem58__c'].str.contains("业管课广发") | Tasks_df2['customItem3__c'].str.contains("业管课广发") | Tasks_df2['customItem3__c'].str.contains("广发型录"))] 

# 爽約
Tasks_df3 = Tasks_df.loc[Tasks_df['customItem3__c'].isin(["K大-客户未上线", "K大-客户末上线"])]
# Tasks_df3 = Tasks_df3.loc[~(Tasks_df3['customItem58__c'].str.contains("已邀"))]

# 未接聽
Tasks_df4 = Tasks_df.loc[Tasks_df['customItem3__c'].isin(["未接听"])] 
# Tasks_df4 = Tasks_df4.loc[~(Tasks_df3['customItem3__c'].str.contains("已邀"))]

today = datetime.now().date()


In [28]:
'''
select from Sales list K大顧問暨電訪人員名單
'''
url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
}
queryLocator = ''
Sales_list = pd.DataFrame()

while True:
    data = {
        "xoql": '''
        SELECT 
        customItem2__c 电访人员, 
        customItem4__c 预估通数, 
        customItem3__c 电访人员类型
        FROM 
            customEntity47__c
        ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    Sales_list = pd.concat([Sales_list, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']

Sales_list['电访人员类型']= Sales_list['电访人员类型'].astype(str)
Sales_list = Sales_list[Sales_list['预估通数']!='']
Sales_list['预估通数']= Sales_list['预估通数'].astype(int)
Sales_list = pd.merge(Sales_list, user_df, left_on = '电访人员', right_on = 'id', how = 'left')
Sales_list = Sales_list[Sales_list['预估通数']>0]
Sales_list1 = Sales_list.loc[~Sales_list['电访人员类型'].str.contains("产品顾问")]
# Sales_list1 = Sales_list1.loc[~Sales_list1['name'].isin(['黃祿庭', '陳鵬宇', '周淑真', '葉子瑄', '黃詩婷', '林育君'])] # 4/11智齒人員

In [29]:
## 已邀約

# # 將今天的日期減去7天
# seven_days_ago = today - timedelta(days=7)

# # 選擇邀約日期早於七天前的行
# Tasks_df2 = Tasks_df2[Tasks_df2['邀約日期'] < seven_days_ago]

Tasks_df2 = Tasks_df2[Tasks_df2['邀約日期'] < today]

Tasks_df2['电访人员'] = np.random.choice(Sales_list1['电访人员'], size=len(Tasks_df2))
Tasks_df2 = pd.merge(Tasks_df2, user_df, left_on = '电访人员', right_on = 'name', how = 'left')
Tasks_df2 = Tasks_df2.drop(['customItem10__c'],axis=1)
Tasks_df2 = Tasks_df2.rename(columns={'dimDepart_x':'dimDepart','电访人员':'customItem10__c', 'id_x':'id', 'name_x':'name'})
Tasks_df2['customItem1__c'] = today
Tasks_df2 = Tasks_df2.drop_duplicates(subset=['customItem42__c'], keep='last')

Tasks_df2

,id,name,customItem3__c,customItem8__c,邀約日期,customItem58__c,customItem50__c,customItem42__c,entityType,dimDepart,區域,approvalStatus,customItem11__c,customItem10__c,id_y,name_y,dimDepart_y,customItem1__c
0,3717793404212694,CTM01385519,已邀K大但时间未定,['等待回应'],2025-03-18,['成功邀约K大'],,2836117151815130,3074631761565155,2794746540529195,中山、珠海,[撤回],2756264141690371,3321172257968944,NaN,NaN,NaN,2025-03-20
1,3728638903259945,CTM01397482,已邀K大但时间未定,['等待回应'],2025-03-19,nan,,2836129559467525,3074631761565155,2794755576436161,上海一,[审批中],2668687488258842,3715951347749679,NaN,NaN,NaN,2025-03-20


In [30]:
## K大爽約
# Tasks_df3 = Tasks_df3[Tasks_df3['邀約日期'] < seven_days_ago]
Tasks_df3 = Tasks_df3[Tasks_df3['邀約日期'] < today]
Tasks_df3['customItem1__c'] = today

Tasks_df3

,id,name,customItem10__c,customItem3__c,customItem8__c,邀約日期,customItem58__c,customItem50__c,customItem42__c,entityType,dimDepart,區域,approvalStatus,customItem11__c,customItem1__c
169219,3728640057053953,CTM01398590,3429999653639632,K大-客户未上线,['等待回应'],2025-03-19,nan,,2836151018509843,3074631761565155,2794748093863381,成都,[审批中],2756264280856084,2025-03-20


In [ ]:
# ## 未接聽
# Tasks_df4 = Tasks_df4[Tasks_df4['邀約日期'] < today]

# Tasks_df4['电访人员'] = np.random.choice(Sales_list1['电访人员'], size=len(Tasks_df4))
# Tasks_df4 = pd.merge(Tasks_df4, user_df, left_on = '电访人员', right_on = 'name', how = 'left')
# Tasks_df4 = Tasks_df4.drop(['customItem10__c'],axis=1)
# Tasks_df4 = Tasks_df4.rename(columns={'dimDepart_x':'dimDepart','电访人员':'customItem10__c', 'id_x':'id', 'name_x':'name'})
# Tasks_df4['customItem1__c'] = today
# Tasks_df4 = Tasks_df4.drop_duplicates(subset=['customItem42__c'], keep='last')

# Tasks_df4

In [ ]:
# # Tasks_df4 = Tasks_df4[~Tasks_df4['approvalStatus'].str.contains("撤回")] 
# Tasks_df4['approvalStatus'] = Tasks_df4['approvalStatus'].astype(str)
# Tasks_df4 = Tasks_df4.loc[~(Tasks_df4['approvalStatus'].str.contains("撤回"))]
# Tasks_df4

In [31]:
'''
from K大預約表
'''

url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
}
queryLocator = ''
K_visit_df = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''
        SELECT 
        name,
        customItem6__c 联系人, 
        customItem2__c 预约日期, 
        customItem40__c, 
        customItem9__c 联系人手机号
        FROM customEntity26__c
        ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    K_visit_df = pd.concat([K_visit_df, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']

# 5/27 美耐板K大開始
K_visit_df['预约日期'] = pd.to_datetime(K_visit_df['预约日期'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Taipei')
start_date = pd.Timestamp('2024-05-27', tz='UTC')
K_visit_df['预约日期'] = pd.to_datetime(K_visit_df['预约日期'])
K_visit_df = K_visit_df[K_visit_df['预约日期'] >= start_date]

# 有舉行K大
K_visit_df['customItem40__c'] = K_visit_df['customItem40__c'].astype(str)
K_visit_df_held = K_visit_df[K_visit_df['customItem40__c'].str.contains("是")]

# 未來場次
today_date = datetime.now().strftime('%Y-%m-%d')
K_visit_df_future = K_visit_df[K_visit_df['预约日期'] >= today_date]

K_visit_df = pd.concat([K_visit_df_held, K_visit_df_future])

/var/folders/sb/5kcp5c2n3fg2g6ywrjr9_f580000gn/T/ipykernel_95660/2172204724.py:37: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  K_visit_df['预约日期'] = pd.to_datetime(K_visit_df['预约日期'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Taipei')


In [32]:
'''
from K大預約参會人
'''
url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
}
queryLocator = ''
K_visit_df_attendant = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''
        select 
        name,
        customItem1__c K大预约表编号,
        customItem2__c 联系人, 
        customItem5__c 联系人手机号
        from customEntity27__c
        ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    K_visit_df_attendant = pd.concat([K_visit_df_attendant, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
    
K_visit_df = pd.concat([K_visit_df, K_visit_df_attendant])


In [33]:
# Tasks_df5 = pd.concat([Tasks_df2,Tasks_df3, Tasks_df4])
Tasks_df5 = pd.concat([Tasks_df2, Tasks_df3])

Tasks_df5 = Tasks_df5.drop('id_y', axis=1)
Tasks_df5 = Tasks_df5.reset_index(drop=True)
Tasks_df5.columns
Tasks_df5 = Tasks_df5.drop('dimDepart_y', axis=1)
Tasks_df5 = pd.merge(Tasks_df5, user_df, left_on = '區域', right_on = 'name', how = 'left')
Tasks_df5 = Tasks_df5.rename(columns={'dimDepart_x':'dimDepart', 'id_y':'ownerId', 'id_x':'id'})

Tasks_df5


,id,name_x,customItem3__c,customItem8__c,邀約日期,customItem58__c,customItem50__c,customItem42__c,entityType,dimDepart,區域,approvalStatus,customItem11__c,customItem10__c,name_y,customItem1__c,ownerId,name_y,dimDepart_y
0,3717793404212694,CTM01385519,已邀K大但时间未定,['等待回应'],2025-03-18,['成功邀约K大'],,2836117151815130,3074631761565155,2794746540529195,中山、珠海,[撤回],2756264141690371,3321172257968944,NaN,2025-03-20,2824643987150336,中山、珠海,2794746540529195
1,3728638903259945,CTM01397482,已邀K大但时间未定,['等待回应'],2025-03-19,nan,,2836129559467525,3074631761565155,2794755576436161,上海一,[审批中],2668687488258842,3715951347749679,NaN,2025-03-20,2957358145311573,上海一,2794755576436161
2,3728640057053953,CTM01398590,K大-客户未上线,['等待回应'],2025-03-19,nan,,2836151018509843,3074631761565155,2794748093863381,成都,[审批中],2756264280856084,3429999653639632,NaN,2025-03-20,2957376513676107,成都,2794748093863381


In [34]:
# 去重 保留第一筆
Tasks_df6 = Tasks_df5.drop_duplicates(subset='customItem42__c', keep='last')

# 重複的資料
duplicates_df = Tasks_df5[Tasks_df5.duplicated(subset='customItem42__c', keep='last')]

duplicates_df

,id,name_x,customItem3__c,customItem8__c,邀約日期,customItem58__c,customItem50__c,customItem42__c,entityType,dimDepart,區域,approvalStatus,customItem11__c,customItem10__c,name_y,customItem1__c,ownerId,name_y,dimDepart_y


In [35]:
K_invite_task = Tasks_df6

# 去除已K大+已邀約
K_invite_task = K_invite_task[~K_invite_task['customItem42__c'].isin(K_visit_df['联系人'].drop_duplicates())]

K_invite_task = K_invite_task[['customItem42__c',
                               'entityType',
                               'customItem1__c',
                               'customItem3__c',
                               'customItem58__c',
                               'customItem10__c',
                               'dimDepart',
                               'customItem11__c',
                               'ownerId',
                               'customItem50__c']]
K_invite_task = K_invite_task.astype(str)
K_invite_task = K_invite_task[K_invite_task['customItem10__c']!='']
K_invite_task['customItem10__c'].value_counts()
K_invite_task['customItem58__c'] = K_invite_task['customItem58__c'].replace('nan', '')
K_invite_task


,customItem42__c,entityType,customItem1__c,customItem3__c,customItem58__c,customItem10__c,dimDepart,customItem11__c,ownerId,customItem50__c
0,2836117151815130,3074631761565155,2025-03-20,已邀K大但时间未定,['成功邀约K大'],3321172257968944,2794746540529195,2756264141690371,2824643987150336,
1,2836129559467525,3074631761565155,2025-03-20,已邀K大但时间未定,,3715951347749679,2794755576436161,2668687488258842,2957358145311573,
2,2836151018509843,3074631761565155,2025-03-20,K大-客户未上线,,3429999653639632,2794748093863381,2756264280856084,2957376513676107,


In [13]:
'''
ask bulk id
'''
url_2 = "https://api-scrm.xiaoshouyi.com/rest/bulk/v2/job"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/json"
}
data = {
     "data": {
        "operation": "insert",####################################################################3
        "object": "customEntity14__c",
        "execOption": ["CHECK_RULE", "CHECK_DUPLICATE"]
    }
    
}
response = requests.post(url_2, headers = headers, json = data)
test = response.json()
bulk_id = test["result"]["id"]
print(bulk_id)


e9fa96a8caa871e0ab36de8779d0f0cfc955b3cc8f77453d8c9b31268239ae8b


In [14]:
'''
insert to crm if dataframe over 5000
'''
batch_size = 5000


num_batches = (len(K_invite_task) + batch_size - 1) // batch_size

all_responses = []

current_batch_index = 0

while current_batch_index < num_batches:
    start_index = current_batch_index * batch_size
    end_index = (current_batch_index + 1) * batch_size
    current_batch_df = K_invite_task.iloc[start_index:end_index]
    json_data = current_batch_df.to_dict(orient='records')
    
    url_2 = "https://api-scrm.xiaoshouyi.com/rest/bulk/v2/batch"
    headers = {
        "Authorization": f"Bearer {ac_token}",
        "Content-Type":"application/json"
    }
    data = {
        "data": {
            "jobId": f"{bulk_id}",
            "datas": json_data
        }
    }
    response = requests.post(url_2, headers=headers, json=data)
    all_responses.append(response)
    current_batch_index += 1
for response in all_responses:
    pass

In [21]:
'''
select from Tasks 
'''
url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"

headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
}

queryLocator = ''

Tasks_df = pd.DataFrame()

while True:
    data = {
        "xoql": '''
        SELECT 
        id,
        name, 
        customItem10__c, 
        customItem3__c, 
        approvalStatus
        FROM customEntity14__c 
        WHERE 
            entityType = '3074631761565155'
        AND
            createdBy = '3052177351988706'
        ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    Tasks_df = pd.concat([Tasks_df, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
    
Tasks_df['approvalStatus']= Tasks_df['approvalStatus'].astype(str)
Tasks_df = Tasks_df.loc[Tasks_df['approvalStatus'].str.contains("待提交")] 

Tasks_df

,id,name,customItem10__c,customItem3__c,approvalStatus
141325,3730048523441647,CTM01401702,3271972840691240,K大-客户未上线,['待提交']
141326,3730048523441648,CTM01401515,3147232434771494,K大-客户未上线,['待提交']
141411,3730048523621857,CTM01401226,3226682698535691,K大-客户未上线,['待提交']
141414,3730048523621860,CTM01401619,3708943322109405,已邀K大但时间未定,['待提交']
141422,3730048523621868,CTM01401627,3063328310809429,K大-客户未上线,['待提交']
141438,3730048524916170,CTM01401527,3226682698535691,K大-客户未上线,['待提交']
141439,3730048524916171,CTM01401528,2787324544751087,K大-客户未上线,['待提交']


In [22]:
'''
submit_task
'''

status_url = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/creekflow/task/actions/preProcessor"
task_url = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/creekflow/task"
headers = {
        "Authorization": f"Bearer {ac_token}",
        "Content-Type": "application/json"
    }

def preProcessor(process_ID):
    status_body = {
        "data": {
            "action": "submit",
            "entityApiKey": "customEntity14__c",
            "dataId": process_ID }}

    response = requests.post(status_url, headers=headers, json=status_body)
    crm_json = response.json()['data']
    return crm_json

tasks_df_last_row = Tasks_df.iloc[-1]
approval_status = preProcessor(tasks_df_last_row['id'])

def submit_task(row):
    data_id = row['id']
    task_id = row['customItem10__c']
    url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/creekflow/task"
    headers = {
        "Authorization": f"Bearer {ac_token}",
        "Content-Type": "application/json"
    }

    data = {
        "data": {
            "action": "submit",
            "entityApiKey": "customEntity14__c",
            "dataId": data_id,
            "procdefId": approval_status['procdefId'],
            "nextTaskDefKey": approval_status['nextTaskDefKey'],
            "nextAssignees": [task_id],
            "ccs": [task_id]  
        }
    }

    response = requests.post(url_2, headers=headers, json=data)
    result = response.json()
    print(f"Response for dataId {data_id}: {result}")

# Assuming data_ids is your DataFrame
data_ids_df = Tasks_df[['id', 'customItem10__c']]

# Create a thread for each row in the DataFrame
threads = []
for index, row in data_ids_df.iterrows():
    thread = threading.Thread(target=submit_task, args=(row,))
    threads.append(thread)
    thread.start()
    time.sleep(0.08)

# Wait for all threads to finish
for thread in threads:
    thread.join()
    

Response for dataId 3730048523441648: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3730048523441647: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3730048523621857: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3730048523621860: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3730048524916170: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3730048523621868: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3730048524916171: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}


In [36]:
'''
get procInstId
'''
Tasks_df2 = Tasks_df6[['id']]
# Tasks_df2 = Tasks_df4[['id']]
base_url = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/creekflow/history/filter?entityApiKey=customEntity14__c&dataId="

def fetch_data(data_id):
    print(data_id)
    api_url = f"{base_url}{data_id}&stageFlg=false"
    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        api_data = response.json()
        if api_data["data"]:
          print(api_data["data"][-1]["procInstId"])
          return json.dumps({ "dataId": data_id, "procInstId": api_data["data"][-1]["procInstId"] })
          # df = pd.DataFrame(api_data["data"][0]["procInstId"], dtype=str)
        else:
            print("Data is empty")
            return
    else:
        print(f"Error accessing API for dataId {data_id}. Status code: {response.status_code}")

        return
    

# Set the maximum number of threads you want to use
max_threads = 2  # You can adjust this based on your needs


with ThreadPoolExecutor(max_threads) as executor:
    # Use executor.map to asynchronously fetch data for each row in parallel
    dfs = list(executor.map(fetch_data, Tasks_df2['id']))
    # Add a time delay between each thread

json_data = [entry for entry in dfs if entry is not None]
Tasks_df2 = pd.json_normalize([json.loads(entry) for entry in json_data])

37177934042126943728638903259945

3717808371652051
3728640057053953
3728641056641825
3728641796068148


In [37]:
'''
withdraw_task
'''

def withdraw_task(row):
    data_id = row['dataId']
    task_id = row['procInstId']
    url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/creekflow/task"
    headers = {
        "Authorization": f"Bearer {ac_token}",
        "Content-Type": "application/json"
    }
    data = {
        "data": {
            "action": "withdraw",
            "entityApiKey": "customEntity14__c",
            "dataId": data_id,
            "procInstId": task_id
        }
    }

    response = requests.post(url_2, headers=headers, json=data)
    result = response.json()
    print(f"Response for dataId {data_id}: {result}")

# Assuming data_ids is your DataFrame
data_ids_df = Tasks_df2[['dataId', 'procInstId']]

# Create a thread for each row in the DataFrame
threads = []
for index, row in data_ids_df.iterrows():
    thread = threading.Thread(target=withdraw_task, args=(row,))
    threads.append(thread)
    thread.start()
    time.sleep(0.08)

# Wait for all threads to finish
for thread in threads:
    thread.join()


Response for dataId 3717793404212694: {'code': '1602011', 'msg': '只有在“处理中”的流程才能撤回', 'data': None, 'errorInfo': None}
Response for dataId 3728638903259945: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3728640057053953: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}


In [38]:
'''
ask bulk id
'''
url_2 = "https://api-scrm.xiaoshouyi.com/rest/bulk/v2/job"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/json"
}
data = {
     "data": {
        "operation": "delete",####################################################################3
        "object": "customEntity14__c",
        "execOption": ["CHECK_RULE", "CHECK_DUPLICATE"]
    }
    
}
response = requests.post(url_2, headers = headers, json = data)
test = response.json()
bulk_id = test["result"]["id"]
print(bulk_id)

Tasks_df3 = Tasks_df2[['dataId']]
Tasks_df3 = Tasks_df3.rename(columns={'dataId':'id'})

e9fa96a8caa871e0ab36de8779d0f0cf9ee453d6014d43f4a51e35044a72f1c1


In [39]:
'''
delete to crm if dataframe over 5000
'''
batch_size = 5000


num_batches = (len(Tasks_df3) + batch_size - 1) // batch_size

all_responses = []

current_batch_index = 0

while current_batch_index < num_batches:
    start_index = current_batch_index * batch_size
    end_index = (current_batch_index + 1) * batch_size
    current_batch_df = Tasks_df3.iloc[start_index:end_index]
    json_data = current_batch_df.to_dict(orient='records')
    
    url_2 = "https://api-scrm.xiaoshouyi.com/rest/bulk/v2/batch"
    headers = {
        "Authorization": f"Bearer {ac_token}",
        "Content-Type":"application/json"
    }
    data = {
        "data": {
            "jobId": f"{bulk_id}",
            "datas": json_data
        }
    }
    response = requests.post(url_2, headers=headers, json=data)
    all_responses.append(response)
    current_batch_index += 1
for response in all_responses:
    pass